In [ ]:
!nvidia-smi

In [ ]:
try:
    import resnest
except ModuleNotFoundError:
    !pip install -q "../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest"

In [ ]:
TOP_BIRDS = {
    'SNE' : ['whcspa','gcrfin','amepip'],
    'SSW' : ['grycat','reevir1'],
    'COR' : ['rucwar'],
}

In [ ]:
import numpy as np
import librosa as lb
import soundfile as sf
import pandas as pd
import cv2
import os
from pathlib import Path
import re
import librosa as lb
import librosa.display as lbd
import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from tqdm.notebook import tqdm
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/vision060/vision-0.6.0')
import timm
import time
from resnest.torch import resnest50, resnest101, resnest50_fast_1s1x64d

In [ ]:
shortest_distances = pd.read_csv('../input/primary-secondary-labels-shortest-distances/shortest_distance.csv',index_col=0).set_index('birds').to_dict()

# Configs

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5
THRESH = 0.25


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

# Data

In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec

In [ ]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

In [ ]:
class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step*self.sr
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        return images
    
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

# Inference

In [ ]:
test_data = BirdCLEFDataset(data=data,step=1)
len(test_data), test_data[0].shape

In [ ]:
def load_net(checkpoint_path, num_classes=NUM_CLASSES):
    if "resnest50" in checkpoint_path:
        net = resnest50(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    if "resnest50_fast_1s1x64d" in checkpoint_path:
        net = resnest50_fast_1s1x64d(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    elif "resnest101" in checkpoint_path:
        net = resnest101(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    elif 'tf_efficientnet_b' in checkpoint_path:
        net = timm.create_model('tf_efficientnet_b0_ns', pretrained = False)
        net.classifier = nn.Linear(net.classifier.in_features,num_classes)
    elif "resnext" in checkpoint_path:
        net = timm.create_model('resnext50_32x4d', pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    elif "densenet121" in checkpoint_path:
        net = getattr(timm.models.densenet, "densenet121")(pretrained=False)
        net.classifier = nn.Linear(net.classifier.in_features,num_classes)
    elif 'seresnet' in checkpoint_path:
        net = getattr(timm.models.resnet, 'seresnet50')(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    net.load_state_dict(d['net'])
    net = net.to(DEVICE)
    net = net.eval()
    return net

def load_public_net(checkpoint_path, num_classes=NUM_CLASSES):
    net = resnest50(pretrained=False)
    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

In [ ]:
checkpoint_paths_resnest = {
    Path('../input/resnestbirdclefnew/resnest50_fold0_e43_augs_mix5.pth') : 1/9,
    Path('../input/resnestbirdclefnew/resnest50_fold1_e29_cont_augs.pth') : 1/9,
    Path('../input/resnestbirdclefnew/resnest50_fold2_e29_augs.pth') : 1/9,
    Path('../input/resnestbirdclefnew/resnest50_fold2_e46_augs_mix5.pth') : 1/9,
    Path('../input/resnestbirdclefnew/resnest50_fold3_e50_augs_mix5_3.pth') : 1/9,
    Path('../input/resnestbirdclefnew/resnest50_fold4_e53_augs_mix5.pth') : 1/9,
    Path('../input/other-models/resnest50_fast_1s1x64d_fold4_e47_cont_pre_augs_mix3_3.pth') : 1/9,
    Path("../input/resnest50add/resnest50_fold1_e50_augs_mix3_3.pth") :1/9,
    Path('../input/resnest50add/resnest50_fold4_e48_augs_mix3_3.pth') :1/9,
}

checkpoint_paths_resnest = {x:checkpoint_paths_resnest[x]*0.6 for x in checkpoint_paths_resnest}
    
    
nets_resnest = {
        load_net(checkpoint_path.as_posix()):checkpoint_paths_resnest[checkpoint_path] for checkpoint_path in checkpoint_paths_resnest
}
sum(checkpoint_paths_resnest.values())

In [ ]:
ls /kaggle/input/kkiller-birdclef-models-public/

In [ ]:
os.path.exists("../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth")

In [ ]:
checkpoint_paths_public = {
    "../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth":0.5
}
nets_public = {
        load_public_net(checkpoint_path):checkpoint_paths_public[checkpoint_path] for checkpoint_path in checkpoint_paths_public
}
sum(checkpoint_paths_public.values())

In [ ]:
# checkpoint_paths_other = {
#         Path('../input/densenet121/densenet121_fold1_e53_cont_augs_mix5.pth') : 1/7,
#         Path('../input/densenet121/densenet121_fold1_e54_cont_augs_mix5.pth') : 1/7,
#         Path('../input/birdclef-efficientnet/resnext50_32x4d_fold2_e54_augs_mix5.pth') : 1/7,
#         Path('../input/resnestbirdclefnew/resnest50_fold1_e45_selfmix5_augs.pth') : 1/7,
#      Path('../input/densenet121/resnest101_fold0_e52_augs_mix3_3.pth') : 1/7,
#     Path('../input/densenet121/resnest101_fold4_e30_augs_mix3_3.pth') : 1/7,
#     Path('../input/other-models/seresnet50_fold3_e46_cont_augs_mix5.pth') : 1/7,
# }

# checkpoint_paths_other = {x:checkpoint_paths_resnest[x]*0.1 for x in checkpoint_paths_other}

# nets_other = {
#         load_net(checkpoint_path.as_posix()):checkpoint_paths_other[checkpoint_path] for checkpoint_path in checkpoint_paths_other
# }
nets_other = {}

In [ ]:
checkpoint_paths_eff = {
    Path('../input/birdclef-efficientnet/tf_efficientnet_b0_ns_fold0_e53_cont_augs_mix5.pth') : 1/3,
    Path('../input/birdclef-efficientnet/tf_efficientnet_b0_ns_fold1_e49_cont_augs_mix5.pth'): 1/3,
    Path('../input/birdclef-efficientnet/tf_efficientnet_b0_ns_fold3_e50_augs_mix5.pth') : 1/3,
}
checkpoint_paths_eff = {x:checkpoint_paths_eff[x]*0.6 for x in checkpoint_paths_eff}

nets_eff = {
        load_net(checkpoint_path.as_posix()):checkpoint_paths_eff[checkpoint_path] for checkpoint_path in checkpoint_paths_eff
}
sum(checkpoint_paths_eff.values())

In [ ]:
len(nets_resnest), len(nets_other), len(nets_eff),  len(nets_public)

In [ ]:
nets = {}
for d in [nets_resnest, nets_other, nets_eff,nets_public]:
    nets.update(d)
len(nets)

In [ ]:
@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

In [ ]:
def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names

In [ ]:
def predict(nets, test_data, names=True):
    preds = []
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            weights_sum = sum([nets[net] for net in nets])
            for net in nets:
                o = net(xb)[:,:NUM_CLASSES]
                o = torch.sigmoid(o)

                pred += o*nets[net]
                
            pred /= weights_sum
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred))
            pred = pd.DataFrame(pred.cpu().numpy()).rename(columns=INV_LABEL_IDS)
            pred['audio_filename'] = test_data.data.loc[idx,'filename']
            pred['seconds'] = pred.index+5
            preds.append(pred)
    return pd.concat(preds).reset_index(drop=True)

In [ ]:
pred_probas = predict(nets, test_data, names=False)
print(len(pred_probas))

In [ ]:
audio_ids = test_data.data.set_index('id').site.to_dict()

In [ ]:
import gc

del nets, nets_resnest, nets_eff, nets_other, nets_public,test_data
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import os
all_birds = os.listdir('../input/birdclef-2021/train_short_audio/')

In [ ]:
oof = pred_probas
oof['audio_id'] = oof.audio_filename.apply(lambda x:x.split('_')[0]).astype(int)
oof['site'] = oof.audio_filename.apply(lambda x:x.split('_')[1])

## Feature Engineering
df_pred = pd.melt(oof,value_vars=all_birds,id_vars=['audio_id','site','seconds'],var_name='bird',value_name='prob').sort_values(['audio_id','bird','seconds']).reset_index(drop=True)
df_pred['rolling_mean_3'] = df_pred.groupby(['audio_id','site','bird']).prob.apply(lambda x:x.rolling(3,center=True,min_periods=1).mean())
# df_pred['rolling_mean_3'] = df_pred.groupby(['audio_id','bird']).prob.rolling(3,center=True,min_periods=1).mean().fillna(
#     df_pred.groupby(['audio_id','bird']).prob.rolling(2,min_periods=1).mean()).values
# df_pred['rolling_mean_5'] = df_pred.groupby(['audio_id','bird']).prob.rolling(5,center=True,min_periods=1).mean().values
# df_pred['rolling_mean_7'] = df_pred.groupby(['audio_id','bird']).prob.rolling(7,center=True,min_periods=1).mean().values
df_pred['rolling_mean_9'] = df_pred.groupby(['audio_id','site','bird']).prob.apply(lambda x:x.rolling(9,center=True,min_periods=1).mean())
# df_pred['rolling_mean_9'] = df_pred.groupby(['audio_id','bird']).prob.rolling(9,center=True,min_periods=1).mean().fillna(
#     df_pred.groupby(['audio_id','bird']).prob.rolling(5,min_periods=1).mean()).values
# df_pred['rolling_mean_15'] = df_pred.groupby(['audio_id','bird']).prob.rolling(15,center=True,min_periods=1).mean().values
# df_pred['rolling_max_15'] = df_pred.groupby(['audio_id','bird']).prob.rolling(15,center=True,min_periods=1).max().values
# df_pred['rolling_max_60'] = df_pred.groupby(['audio_id','bird']).prob.rolling(60,center=True,min_periods=1).max().values
# df_pred['rolling_max_600'] = df_pred.groupby(['audio_id','bird']).prob.rolling(600,center=True,min_periods=1).max().fillna(method='ffill').values
max_probabilities = df_pred.groupby(['audio_id','site','bird']).prob.max().reset_index().rename(columns={'prob':'max_prob'})
df_pred = df_pred[df_pred.seconds%5==0].reset_index(drop=True)
df_pred = pd.merge(df_pred,max_probabilities,on=['audio_id','bird','site'])
df_pred['min_distance'] = df_pred.apply(lambda row: shortest_distances[f"{row.site}_distance"][row.bird],axis=1)
print(df_pred.shape)
df_pred['min_distance'] = df_pred.apply(lambda row: shortest_distances[f"{row.site}_distance"][row.bird],axis=1)
df_pred = df_pred.loc[df_pred['min_distance']<100]
print(df_pred.shape)
df_pred['min_distance'] = df_pred['min_distance']/4000

In [ ]:
features = ['rolling_mean_3','rolling_mean_9','max_prob', 'min_distance']

In [ ]:
train = pd.read_csv('../input/birdclef-svc-ensemble-optimisation/df.csv')

In [ ]:
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
train = train[train.prob>0.01]
clf = SVC(gamma='auto',probability=True)
clf.fit(train[features], train.target)
# clf = LogisticRegression(random_state=0).fit(train[features], train.target)
df_pred['adjusted_prediction'] = clf.predict_proba(df_pred[features])[:,1]

In [ ]:
del train,clf
gc.collect()

In [ ]:
template = df_pred[df_pred.site.isin(['COL','COR'])][['audio_id','site','seconds']].drop_duplicates().reset_index(drop=True)

In [ ]:
df_pred_subset = df_pred[df_pred.site.isin(['COL','COR'])].reset_index(drop=True)
threshold1 = 0.14
threshold2 = 0.34
# df_pred_subset.loc[df_pred_subset.bird=='grhowl','adjusted_prediction'] = 0
# df_pred_subset.loc[df_pred_subset.bird=='plupig2','adjusted_prediction'] = 0
df_pred_subset = df_pred_subset.sort_values(['audio_id','seconds','adjusted_prediction']).reset_index(drop=True).groupby(['audio_id','seconds']).tail(3)
nocall_prob = (1-df_pred_subset.groupby(['audio_id','site','seconds']).adjusted_prediction.max()).reset_index()
nocall_prob['bird'] = 'nocall'
subset = df_pred_subset#[(df_pred_subset.prob>0.005)]
for site in TOP_BIRDS:
        subset.loc[(subset.site==site) & (subset.bird.isin(TOP_BIRDS[site])),'adjusted_prediction'] += 0.1
subset = subset[subset.adjusted_prediction>threshold1]
subset2 = nocall_prob[nocall_prob.adjusted_prediction>threshold2]
subset = pd.concat([subset,subset2]).groupby(['audio_id','seconds']).bird.apply(lambda x:" ".join(x)).reset_index()
subset = pd.merge(template,
                  subset,on=['audio_id','seconds'],how='outer').fillna('nocall').rename(columns={'bird':'birds'})
subset['row_id'] = subset.apply(lambda row:f"{row.audio_id}_{row.site}_{row.seconds}",axis=1)

In [ ]:
subset[['row_id','birds']].to_csv('submission_0.csv',index=False)
subset.shape

In [ ]:
df_pred = df_pred[df_pred.site.isin(['SSW','SNE'])].reset_index(drop=True)

In [ ]:
def compute_f1(row):
    birds1 = set(row.birds_x.split(' '))
    birds2 = set(row.birds_y.split(' '))
    intersection = birds1.intersection(birds2)
    return 2*len(intersection)/(len(birds1)+len(birds2))

ground_truth = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')

def cv_score(submission):
    df = pd.merge(ground_truth,submission,on='row_id')
    df['f1'] = df.apply(compute_f1,axis=1)
    print(df.birds_y.value_counts(1).head(5).to_dict())
    print("CV-> ","Net:",df.f1.mean(),"NoCall:",df[df.birds_x=='nocall'].f1.mean(),"Call:",df[df.birds_x!='nocall'].f1.mean())

    print('Alt CV: ', df[df.birds_x=='nocall'].f1.mean()*0.54+df[df.birds_x!='nocall'].f1.mean()*0.46)

In [ ]:
import os
IS_SUBMIT = len([x for x in os.listdir('../input/birdclef-2021/test_soundscapes/') if x.endswith('.ogg')])>0
print(IS_SUBMIT)

In [ ]:
if not IS_SUBMIT:
    print("CV For COL & COR")
    cv_score(subset.loc[subset.site.isin(['COL','COR']),['row_id','birds']])
    print()
    print("CV For all sites")
    cv_score(subset[['row_id','birds']])

In [ ]:
del df_pred_subset,oof, subset, subset2, nocall_prob
gc.collect()

In [ ]:
# !python '../input/birdclef-cornell-birdcall-1st-place-script/first_place_inference_ssw_sne.py'

In [ ]:
import os
import pandas as pd
from shutil import copyfile

!nvidia-smi

!python '../input/birdclef-cornell-1st-place-script-modified/first_place_inference_ssw_sne.py'

In [ ]:
oofs = pd.read_csv('predictions.csv')
predictions = [col for col in oofs.columns if 'prob' in col]
frame_predictions = [col for col in oofs.columns if 'prob' in col if 'clip' not in col]
clip_predictions = [col for col in oofs.columns if 'prob' in col if 'clip' in col]
oofs['clip_avg'] = oofs[clip_predictions].mean(axis=1)
oofs['frame_avg'] = oofs[frame_predictions].mean(axis=1)
oofs['end_seconds'] = ((oofs.seconds//5)+1)*5
oofs_aggregated_mean = oofs.groupby(['audio_id','site','bird','end_seconds'])[predictions].mean().reset_index().rename(columns={'end_seconds':'seconds'})
oofs_aggregated_max = oofs.groupby(['audio_id','site','bird','end_seconds'])[frame_predictions].max().reset_index().rename(columns={'end_seconds':'seconds'})
oofs_aggregated_max.columns = [ x+'_max' if 'prob' in x else x for x in oofs_aggregated_max.columns]
oofs_aggregated = pd.merge(oofs_aggregated_max,oofs_aggregated_mean,on=['audio_id','site','bird','seconds'])
max_predictions = [col for col in oofs_aggregated.columns if 'prob' in col if 'max' in col]
agg_predictions = [col for col in oofs_aggregated.columns if 'prob' in col]
max_predictions_selected = [x for i,x in enumerate(max_predictions) if i not in [4]]
clip_predictions_selected = [x for i,x in enumerate(clip_predictions) if i not in [4]]
oofs_aggregated["clip_avg"] = oofs_aggregated[clip_predictions_selected].mean(axis=1)
oofs_aggregated["max_avg"] = oofs_aggregated[max_predictions_selected].mean(axis=1)
oofs_aggregated = oofs_aggregated[oofs_aggregated.clip_avg>0.02]
oofs_aggregated_max = oofs_aggregated.set_index(['audio_id','site','bird','seconds']).to_dict()['max_avg']

In [ ]:
template = df_pred[df_pred.site.isin(['SSW','SNE'])][['audio_id','site','seconds']].drop_duplicates().reset_index(drop=True)
df_pred_subset = df_pred[df_pred.site.isin(['SSW','SNE']) & (df_pred.adjusted_prediction>0.02)]
df_pred_subset['first_place'] = 0
for i,row in tqdm(df_pred_subset.iterrows()):
    if (row.audio_id,row.site,row.bird,row.seconds) in oofs_aggregated_max:
        df_pred_subset.loc[i,'first_place'] = oofs_aggregated_max[(row.audio_id,row.site,row.bird,row.seconds)]

In [ ]:
del oofs,oofs_aggregated_mean,oofs_aggregated_max,oofs_aggregated

In [ ]:
gc.collect()

In [ ]:
!rm -r /tmp/test_audio

In [ ]:
!python '../input/birdclef-2nd-place-inference-1-sec-script/second_place_inference_ssw_sne.py'

In [ ]:
oofs = pd.read_csv('predictions_1.csv')
ground_truth = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')
ground_truth = ground_truth[ground_truth.site=='SSW'].reset_index(drop=True)
predictions = [col for col in oofs.columns if 'prob' in col]
frame_predictions = [col for col in oofs.columns if 'prob' in col if 'clip' not in col]
oofs['frame_avg'] = oofs[frame_predictions].mean(axis=1)
oofs['end_seconds'] = ((oofs.seconds//5)+1)*5
oofs_aggregated_mean = oofs.groupby(['audio_id','site','bird','end_seconds'])[predictions].mean().reset_index().rename(columns={'end_seconds':'seconds'})
oofs_aggregated_max = oofs.groupby(['audio_id','site','bird','end_seconds'])[frame_predictions].max().reset_index().rename(columns={'end_seconds':'seconds'})
oofs_aggregated_max.columns = [ x+'_max' if 'prob' in x else x for x in oofs_aggregated_max.columns]
oofs_aggregated = pd.merge(oofs_aggregated_max,oofs_aggregated_mean,on=['audio_id','site','bird','seconds'])
max_predictions = [col for col in oofs_aggregated.columns if 'prob' in col if 'max' in col]
agg_predictions = [col for col in oofs_aggregated.columns if 'prob' in col]
print(predictions)
max_predictions_selected = [x for i,x in enumerate(max_predictions) if i not in [4]]
frame_predictions_selected = [x for i,x in enumerate(frame_predictions) if i not in [4]]
oofs_aggregated["frame_avg"] = oofs_aggregated[frame_predictions_selected].mean(axis=1)
oofs_aggregated_max = oofs_aggregated.set_index(['audio_id','site','bird','seconds']).to_dict()['frame_avg']

In [ ]:
df_pred_subset['second_place'] = 0
for i,row in tqdm(df_pred_subset.iterrows()):
    if (row.audio_id,row.site,row.bird,row.seconds) in oofs_aggregated_max:
        df_pred_subset.loc[i,'second_place'] = oofs_aggregated_max[(row.audio_id,row.site,row.bird,row.seconds)]

In [ ]:
df_pred_subset.to_csv('df_pred_subset.csv')

In [ ]:
df_pred_subset['ensemble'] = 0.35*df_pred_subset['adjusted_prediction'] + 0.15*df_pred_subset["first_place"] + 0.5*df_pred_subset["second_place"]

In [ ]:
threshold1 = 0.15
threshold2 = 0.7
# df_pred_subset.loc[df_pred_subset.bird=='grhowl','ensemble'] = 0
# df_pred_subset.loc[df_pred_subset.bird=='plupig2','ensemble'] = 0
df_pred_subset = df_pred_subset.sort_values(['audio_id','seconds','ensemble']).reset_index(drop=True).groupby(['audio_id','seconds']).tail(3)
nocall_prob = (1-df_pred_subset.groupby(['audio_id','site','seconds']).ensemble.max()).reset_index()
nocall_prob['bird'] = 'nocall'
subset = df_pred_subset#[(df_pred_subset.prob>0.005)]
for site in TOP_BIRDS:
        subset.loc[(subset.site==site) & (subset.bird.isin(TOP_BIRDS[site])),'ensemble'] += 0.1
subset = subset[subset.ensemble>threshold1]
subset2 = nocall_prob[nocall_prob.ensemble>threshold2]
subset = pd.concat([subset,subset2]).groupby(['audio_id','seconds']).bird.apply(lambda x:" ".join(x)).reset_index()
subset = pd.merge(template,
                  subset,on=['audio_id','seconds'],how='outer').fillna('nocall').rename(columns={'bird':'birds'})
subset['row_id'] = subset.apply(lambda row:f"{row.audio_id}_{row.site}_{row.seconds}",axis=1)

In [ ]:
# df_pred_subset = df_pred[df_pred.site.isin(['COL','COR'])].reset_index(drop=True)
# threshold1 = 0.14
# threshold2 = 0.34

# df_pred_subset = df_pred_subset.sort_values(['audio_id','seconds','adjusted_prediction']).reset_index(drop=True).groupby(['audio_id','seconds']).tail(3)
# nocall_prob = (1-df_pred_subset.groupby(['audio_id','site','seconds']).adjusted_prediction.max()).reset_index()
# nocall_prob['bird'] = 'nocall'
# subset = df_pred_subset[(df_pred_subset.prob>0.005)].copy(deep=True)
# for site in TOP_BIRDS:
#         subset.loc[(subset.site==site) & (subset.bird.isin(TOP_BIRDS[site])),'adjusted_prediction'] += 0.1
# subset = subset[subset.adjusted_prediction>threshold1]
# subset2 = nocall_prob[nocall_prob.adjusted_prediction>threshold2]
# subset = pd.concat([subset,subset2]).groupby(['audio_id','seconds']).bird.apply(lambda x:" ".join(x)).reset_index()
# subset = pd.merge(df_pred_subset[['audio_id','site','seconds']].drop_duplicates(),
#                   subset,on=['audio_id','seconds'],how='outer').fillna('nocall').rename(columns={'bird':'birds'})
# subset['row_id'] = subset.apply(lambda row:f"{row.audio_id}_{row.site}_{row.seconds}",axis=1)

In [ ]:
if not IS_SUBMIT:
    print("CV For SNE & SSW")
    cv_score(subset.loc[subset.site.isin(['SNE','SSW']),['row_id','birds']])
    print()
    print("CV For all sites")
    cv_score(subset[['row_id','birds']])

In [ ]:
subset[['row_id','birds']].to_csv('submission_1.csv',index=False)
subset.shape

In [ ]:
def compute_f1(row):
    birds1 = set(row.birds_x.split(' '))
    birds2 = set(row.birds_y.split(' '))
    intersection = birds1.intersection(birds2)
    return 2*len(intersection)/(len(birds1)+len(birds2))

ground_truth = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')

def cv_score(submission):
    df = pd.merge(ground_truth,submission,on='row_id')
    df['f1'] = df.apply(compute_f1,axis=1)
    print(df.birds_y.value_counts(1).head(5).to_dict())
    print("CV-> ","Net:",df.f1.mean(),"NoCall:",df[df.birds_x=='nocall'].f1.mean(),"Call:",df[df.birds_x!='nocall'].f1.mean())

    print('Alt CV: ', df[df.birds_x=='nocall'].f1.mean()*0.54+df[df.birds_x!='nocall'].f1.mean()*0.46)

In [ ]:
subset0 = pd.read_csv('submission_0.csv')
subset1 = pd.read_csv('submission_1.csv')
subset = pd.concat([subset0,subset1]).reset_index(drop=0)

In [ ]:
if not IS_SUBMIT:
    print("CV For COL & COR")
    cv_score(subset0[['row_id','birds']])
    print()
    print("CV For SNE & SSW")
    cv_score(subset1[['row_id','birds']])
    print()
    print("CV For all sites")
    cv_score(subset[['row_id','birds']])

In [ ]:
subset[['row_id','birds']].to_csv('submission.csv',index=False)
subset.shape